In [45]:
import math
import numpy as np
import pandas as pd
import time
import os
import sys
import tensorflow.compat.v1 as tf

In [28]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [12]:
!apt-get install bedtools

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  bedtools
0 upgraded, 1 newly installed, 0 to remove and 37 not upgraded.
Need to get 577 kB of archives.
After this operation, 2,040 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 bedtools amd64 2.26.0+dfsg-5 [577 kB]
Fetched 577 kB in 2s (333 kB/s)
Selecting previously unselected package bedtools.
(Reading database ... 155222 files and directories currently installed.)
Preparing to unpack .../bedtools_2.26.0+dfsg-5_amd64.deb ...
Unpacking bedtools (2.26.0+dfsg-5) ...
Setting up bedtools (2.26.0+dfsg-5) ...


In [29]:
! wget http://hgdownload.cse.ucsc.edu/admin/exe/linux.x86_64/twoBitToFa
! chmod ugo+rwx twoBitToFa



--2021-12-02 19:28:20--  http://hgdownload.cse.ucsc.edu/admin/exe/linux.x86_64/twoBitToFa
Resolving hgdownload.cse.ucsc.edu (hgdownload.cse.ucsc.edu)... 128.114.119.163
Connecting to hgdownload.cse.ucsc.edu (hgdownload.cse.ucsc.edu)|128.114.119.163|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9367560 (8.9M)
Saving to: ‘twoBitToFa.1’

twoBitToFa.1        100%[===================>]   8.93M  5.99MB/s    in 1.5s    

2021-12-02 19:28:22 (5.99 MB/s) - ‘twoBitToFa.1’ saved [9367560/9367560]



In [30]:
H3F3A_DOHH2_intersect_with_DeepZ = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/H3F3A_DOHH2.intersect_with_DeepZ.bed', sep='\t', names=['chr', 'start', 'end'])
H3F3A_DOHH2_intersect_with_DeepZ['len'] = H3F3A_DOHH2_intersect_with_DeepZ['end'] - H3F3A_DOHH2_intersect_with_DeepZ['start']
positive1 = H3F3A_DOHH2_intersect_with_DeepZ[H3F3A_DOHH2_intersect_with_DeepZ['len'] == 10]

In [31]:
sz = positive1['len'] // 2
positive1['center'] = positive1['start'] + sz
positive1['start'] = positive1['center'] - 100
positive1['end'] = positive1['center'] + 100

positive1 = positive1.drop(columns=['len', 'center'])
positive1 = positive1.drop_duplicates()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

In [32]:
for i in range(positive1.shape[0]):
    start = positive1.iloc[i, :]['start']
    end = positive1.iloc[i, :]['end']
    name = positive1.iloc[i, :]['chr']
    file_name = str(i) + 'z.fa'
    !./twoBitToFa http://hgdownload.cse.ucsc.edu/gbdb/hg19/hg19.2bit $file_name -seq=$name -start=$start -end=$end

In [33]:
!cat *z.fa > pos.fa

In [34]:
cd /content/drive/MyDrive/Colab Notebooks

/content/drive/MyDrive/Colab Notebooks


In [35]:
!bedtools subtract -a Genome.bed -b DeepZ.bed > without_deepz.bed

In [36]:
!bedtools subtract -a without_deepz.bed -b H3F3A_DOHH2.merge.hg19.bed > negative1.bed

In [37]:
negative1 = pd.read_csv('negative1.bed', sep='\t')

In [38]:
negative1.columns = ['chr', 'start', 'end', '4', '5', '6', '7', '8', '9', '10', '11', '12']
negative1.loc[negative1['end'] - negative1['start'] >= 200, 'end'] = negative1['start'] + 200

In [39]:
negative1 = negative1.loc[negative1['end'] - negative1['start'] == 200].sample(10000)


In [40]:
negative1[["chr", "start", "end", "4"]].to_csv('negative.bed', index=None, sep="\t", header=None)

In [23]:
#negative1[["chr", "start", "end", "8"]].to_csv('negative.bed', index=None, sep="\t", header=None)

chr1	174285404	174285604	NM_001366446.1
chr18	46920888	46921088	NM_001353211.3
chr1	179712425	179712625	NM_173509.3
chr6	36765161	36765361	NR_164866.1
chr8	53263939	53264139	NM_001352826.2
chr8	130854495	130854695	NM_001353251.1
chr19	46218830	46219030	NM_001080469.2
chr20	45846756	45846956	NM_001281783.3
chr2	25637788	25637988	NM_001256308.2
chr5	133451550	133451750	NM_201632.5


In [41]:
!wget http://hgdownload.cse.ucsc.edu/admin/exe/linux.x86_64/twoBitToFa
!chmod a+x twoBitToFa

--2021-12-02 19:29:22--  http://hgdownload.cse.ucsc.edu/admin/exe/linux.x86_64/twoBitToFa
Resolving hgdownload.cse.ucsc.edu (hgdownload.cse.ucsc.edu)... 128.114.119.163
Connecting to hgdownload.cse.ucsc.edu (hgdownload.cse.ucsc.edu)|128.114.119.163|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9367560 (8.9M)
Saving to: ‘twoBitToFa.2’

twoBitToFa.2        100%[===================>]   8.93M  6.02MB/s    in 1.5s    

2021-12-02 19:29:24 (6.02 MB/s) - ‘twoBitToFa.2’ saved [9367560/9367560]



In [42]:
!./twoBitToFa http://hgdownload.cse.ucsc.edu/gbdb/hg19/hg19.2bit -bed=negative.bed neg.fa


In [43]:
!head neg.fa

>NM_153348.3
CCCCAACTAAAAAGTTCTTATCCTCAGTGCTGTTCATTTTTTCATGTTAT
TCGTGAATTAGCAGGCCTCCACTTACGAACCTTTTGAAATGATGCCTGTG
TATACTAGAACAACTTGTGACTTACATGCGGTCTTTGTTGGAACGGTCTG
CTGCTTGAAGCAGCTTGGGTttcattcatatcccacaaatatttattaag
>NM_001354297.2
ATTCCTTCAGTTCTTCAAAATCTATTTAGTTGGTAAGTGGATTGTTTAGG
Cttaataaattaatataaattacttgaaattcagtaatatttgatatttt
aattatgtaatattttaGGGATACAATGTAAATACTCAAGGCTCAAGGAA
CTTAGAATAGGGAAAGAATAGTACACAATTGATTTAAAGTATCATGAAAG


**Обучаем модель**

In [ ]:
__author__ = 'jasperz'

In [46]:
# An object of this class represents a neural network, which you can build, print, train, evaluate, save and load.
# Below, the functions are discussed in detail.
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
class NetworkModel:
    tf.disable_v2_behavior()

    # The constructor function
    def __init__(self, file_to_load = None):
        tf.reset_default_graph()
        self.all_layers = []

        gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.1)
        config = tf.ConfigProto(gpu_options=gpu_options)
        config.gpu_options.allow_growth = True

        self.sess = tf.Session(config=config)

        if not file_to_load:
            self.X_placeholder = tf.placeholder(tf.float32, [None, 200, 4],name='X_placeholder')
            self.Y_placeholder = tf.placeholder(tf.float32, [None, 2],name='Y_placeholder')
            self.loaded = False
            self.nn = None
        else:
            self.loaded = True
            self._loadNetworkParameters('models/'+file_to_load)
            self.X_placeholder = tf.get_default_graph().get_tensor_by_name('X_placeholder:0')
            self.Y_placeholder = tf.get_default_graph().get_tensor_by_name('Y_placeholder:0')
            self.predictions_softmax = tf.get_default_graph().get_tensor_by_name('softmax_prediction:0')

    # Adding the input layer
    def addInputLayer(self):
        assert len(self.all_layers) == 0, 'The input layer should be the first layer of the network, and can only be added once.'
        self.all_layers.append(('Input layer','',self.X_placeholder))

    # Adding a convolution layer
    def addConvLayer(self, num_of_filters, filter_width, zero_padding = True):
        assert len(self.all_layers) > 0 and self.all_layers[0][0].startswith('Input layer')
        assert zero_padding in (True,False), 'zero_padding should be True or False (boolean)'
        assert 0 < num_of_filters < 500, 'The number of filters specified should be a positive number, smaller than 500'
        assert 0 < filter_width < 64, 'The width of your filters should be a positive number, smaller than 64'
        assert len(self.all_layers)+1 < 21, 'The total amount of layers should be at most 20'
        assert 'Fully-connected layer' not in [typ for typ,_,_ in self.all_layers], 'You cannot add a convolutional layer after a fully-connected layer'
        assert 'Softmax (output) layer' not in [typ for typ,_,_ in self.all_layers], 'You cannot add a convolutional layer after a softmax layer'
        prev_width = self.all_layers[-1][-1].shape[1]
        assert zero_padding or prev_width >= filter_width, 'You cannot add a (non-zeropadded) convolution of width {} when the previous layer has an output width of {}'.format(filter_width,prev_width)
        self.all_layers.append(('Convolutional layer',
                                '{} filters, width {}, {}zero padding, with ReLU'.format(num_of_filters,
                                                                                         filter_width,
                                                                                         'no ' if not zero_padding else ''),
                                tf.layers.conv1d(self.all_layers[-1][-1],
                                                 filters=num_of_filters,
                                                 kernel_size=filter_width,
                                                 activation=tf.nn.relu,
                                                 padding='same' if zero_padding else 'valid')))

    # Adding a max pooling layer
    def addMaxPoolLayer(self, pool_size):
        assert len(self.all_layers) > 0 and self.all_layers[0][0].startswith('Input layer')
        assert 'Fully-connected layer' not in [typ for typ,_,_ in self.all_layers], 'You cannot add a pooling layer after a fully-connected layer'
        assert 'Softmax (output) layer' not in [typ for typ,_,_ in self.all_layers], 'You cannot add a pooling layer after a softmax layer'
        assert len(self.all_layers)+1 < 21, 'The total amount of layers should be at most 20'
        assert 0 < pool_size < 50, 'The pool size should be lower than 50'
        prev_width = self.all_layers[-1][-1].shape[1]
        assert prev_width >= pool_size, 'You cannot add a pooling layer with pool size {} when the previous layer has an output width of {}'.format(pool_size,prev_width)
        self.all_layers.append(('Max pooling layer',
                                'pool size {}'.format(pool_size),
                                tf.layers.max_pooling1d(self.all_layers[-1][-1],
                                                        pool_size=pool_size,
                                                        strides=pool_size)))

    # Adding a fully-connected layer
    def addFullyConnectedLayer(self,num_of_neurons):
        assert len(self.all_layers) > 0 and self.all_layers[0][0].startswith('Input layer')
        assert 'Softmax (output) layer' not in [typ for typ,_,_ in self.all_layers], 'You cannot add a fully-connected layer after a softmax layer'
        assert len(self.all_layers)+1 < 21, 'The total amount of layers should be at most 20'
        assert 0 < num_of_neurons < 1000, 'The amount of neurons in this layer should be a positive number, lower than 2000'
        if len(self.all_layers[-1][-1].shape) > 2:
            self.all_layers.append(('Flatten layer',
                                    '',
                                   tf.layers.flatten(self.all_layers[-1][-1])))
        self.all_layers.append(('Fully-connected layer',
                                '{} neurons, with ReLU'.format(num_of_neurons),
                                tf.layers.dense(self.all_layers[-1][-1],num_of_neurons)))

    # Adding an output layer
    def addOutputLayer(self):
        assert len(self.all_layers) > 0 and self.all_layers[0][0].startswith('Input layer')
        assert 'Softmax (output) layer' not in [typ for typ,_,_ in self.all_layers], 'You cannot add a softmax (output) layer after a softmax layer'
        assert len(self.all_layers)+1 < 21, 'The total amount of layers should be at most 20'
        if len(self.all_layers[-1][-1].shape) > 2:
            self.all_layers.append(('Flatten layer',
                                    '',
                                    tf.contrib.layers.flatten(self.all_layers[-1][-1])))
        # assert no output layer yet
        # assert # of layers
        self.all_layers.append(('Softmax (output) layer',
                                '2 neurons',
                                tf.layers.dense(self.all_layers[-1][-1], 2,name='logits')))

    # Printing out an overview of the layers
    def printDetails(self):
        print('####################################')
        print('Network information:')
        # count all parameters:
        total_parameters = 0
        # iterating over all variables
        for variable in tf.trainable_variables():
            local_parameters = 1
            shape = variable.get_shape()  # getting shape of a variable
            for i in shape:
                local_parameters *= i.value  # mutiplying dimension values
            total_parameters += local_parameters
        print('This network has {} trainable parameters.'.format(total_parameters))

        for i,(name,info,l) in enumerate(self.all_layers):
            try:
                print('{: >2d}. {:23} {:50} -> Output size: {}'.format(i, name, info, l.shape))
            except AttributeError:
                pass
        print('')
        print('####################################')

    # Function to train the network
    def train(self, trainX, trainY, validX, validY, n_epochs):
        print('####################################')
        assert 'Input layer' in [typ for typ,_,_ in self.all_layers], 'You cannot train a model without an input layer'
        assert 'Softmax (output) layer' in [typ for typ,_,_ in self.all_layers], 'You cannot train a model without an output layer'
        assert self.loaded == False, 'You can not (re)train a model loaded from a file.'
        assert 1 < n_epochs < 100, 'The number of epochs should be greater than 1 and lower than 100'
        assert all(type(l) == list for l in (trainX, trainY, validX, validY)), 'trainX, trainY, validX and validY should all be lists'
        assert all(len(l) > 0 for l in (trainX, trainY, validX, validY)), 'trainX, trainY, validX and validY should not be empty'

        assert len(trainX) == len(trainY), 'trainX and trainY should have the same amount of samples'
        assert len(trainX[0]) == 200 and len(trainX[0][0]) == 4 and type(trainX[0][0][0]) == int, 'trainX should have size (_, 200, 4) and should contain integers'
        assert type(trainY[0]) == int, 'trainY should have length n (for n sequences) and should contain integers'

        assert len(validX) == len(validY), 'validX and validY should have the same amount of samples'
        assert len(validX[0]) == 200 and len(validX[0][0]) == 4 and type(validX[0][0][0]) == int, 'validX should have size (_, 200, 4) and should contain integers'
        assert type(validY[0]) == int, 'validY should have length n (for n sequences) and should contain integers'
        # assert input and output layer
        self._prepare_training()

        self.sess.run(tf.global_variables_initializer())
        self.sess.run(tf.local_variables_initializer())
        train_dataset = _Dataset(trainX, trainY)
        valid_dataset = _Dataset(validX, validY)
        self._printOutputClasses(train_dataset,'training')
        self._printOutputClasses(valid_dataset,'validation')

        best_valid_score = 999999
        print()
        print(' {:^5} | {:^14} | {:^14} | {:^11} | {:^11} | {:^8} '.format('epoch','train cost','valid cost','train acc','valid acc','time'))
        print('-{:-^6}+{:-^16}+{:-^16}+{:-^13}+{:-^13}+{:-^9}-'.format('','','','','',''))

        tr_cost, tr_acc = self._evaluateSet(train_dataset)
        va_cost, va_acc = self._evaluateSet(valid_dataset)
        print(' {:5d} |   {:2.8f}   |   {:2.8f}   |  {:1.7f}  | {:1.7f}  | {:4.2f}s '.format(0,tr_cost,tr_acc,va_cost,va_acc,0))

        for epoch in range(1,n_epochs+1):
            epoch_start_time = time.time()
            epoch_finished = False
            while not epoch_finished:
                batch_x, batch_y, epoch_finished = train_dataset.next_batch(256)
                self.sess.run(self.train_op, feed_dict={self.X_placeholder: batch_x, self.Y_placeholder: batch_y})
            tr_cost, tr_acc = self._evaluateSet(train_dataset)
            va_cost, va_acc = self._evaluateSet(valid_dataset)

            if va_cost < best_valid_score:
                best_valid_score = va_cost
                message = '-> model selected'
                self._storeNetworkParameters('models/tmp')
            else:
                message = ''
            print(' {:5d} |   {:2.8f}   |   {:2.8f}   |  {:1.7f}  | {:1.7f}  | {:4.2f}s {}'.format(epoch,tr_cost,va_cost,tr_acc,va_acc,time.time()-epoch_start_time,message))

        self._loadNetworkParameters('models/tmp')
        print('Finished training')
        print('####################################')

    # Function to generate predictions for a certain dataset.
    def generatePredictions(self, testX):
        assert len(testX[0]) == 200 and len(testX[0][0]) == 4 and type(testX[0][0][0]) == int, 'testX should have size (_, 200, 4) and should contain integers'
        assert self.loaded or 'Input layer' in [typ for typ,_,_ in self.all_layers], 'You cannot test a model without an input layer'
        assert self.loaded or 'Softmax (output) layer' in [typ for typ,_,_ in self.all_layers], 'You cannot test a model without an output layer'
        # assert input and output layer
        all_preds = []
        for i in range(math.ceil(len(testX)/256)):
            batch_x = np.asarray(testX[i*256:(i+1)*256])
            preds = self.sess.run(self.predictions_softmax,feed_dict={self.X_placeholder:batch_x})
            for i in range(len(preds)):
                all_preds.append((preds[i][0],preds[i][1]))
        return all_preds

    # Function to save the model
    def saveModel(self, file_to_save_to):
        assert 'Input layer' in [typ for typ,_,_ in self.all_layers], 'You cannot save a model without an input layer'
        assert 'Softmax (output) layer' in [typ for typ,_,_ in self.all_layers], 'You cannot save a model without an output layer'
        # assert input and output layer
        assert not self.loaded, 'You cannot save a loaded model again.'
        self._storeNetworkParameters('models/'+file_to_save_to)

    def _prepare_training(self):
        # assert all layers -1 == output layer
        gs = tf.train.get_or_create_global_step()
        self.predictions_softmax = tf.nn.softmax(self.all_layers[-1][-1],name='softmax_prediction')

        self.cost_f = tf.losses.softmax_cross_entropy(onehot_labels=self.Y_placeholder, logits=self.all_layers[-1][-1])
        self.optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
        self.train_op = self.optimizer.minimize(loss=self.cost_f,global_step=gs)

        self.acc_f, self.acc_op = tf.metrics.accuracy(labels=tf.argmax(self.Y_placeholder, axis=1),predictions=tf.argmax(self.predictions_softmax, axis=1),name='metric_acc')
        self.metric_var_initializer = tf.variables_initializer(var_list=tf.get_collection(tf.GraphKeys.LOCAL_VARIABLES, scope='metric'))

    def _evaluateSet(self, dataset):
        self.sess.run(self.metric_var_initializer)
        costs = []
        batches_done = False
        while not batches_done:
            batch_x, batch_y, epoch_finished = dataset.next_batch(256)

            cost_batch = self.sess.run(self.cost_f, feed_dict={self.X_placeholder: batch_x,self.Y_placeholder: batch_y})
            _ = self.sess.run([self.acc_op], feed_dict={self.X_placeholder: batch_x,self.Y_placeholder: batch_y})
            costs.extend([cost_batch] * len(batch_y))

            if epoch_finished:
                batches_done = True

        accuracy = self.sess.run([self.acc_f])[0]
        return np.average(costs),accuracy

    def _printOutputClasses(self, dataset, label):
        print()
        counts = dataset.getClassCounts()
        print('Number of {} examples: {}'.format(label,int(np.sum(counts))))
        if len(counts) > 1:
            print('Distribution of the {} set:'.format(label))
            for i in range(min(10,len(counts))):
                print('  # elements of class {} = {}'.format(i,int(counts[i])))

    def _storeNetworkParameters(self, saveToDir):
        try:
            saver = tf.train.Saver()
            if not os.path.exists(saveToDir):
                os.makedirs(saveToDir)
            saver.save(self.sess,saveToDir+'/'+saveToDir[saveToDir.rfind('/')+1:])
        except Exception:
            print('SOMETHING WENT WRONG WITH STORING SHIT JASPER!! ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
            print(sys.exc_info())
            print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')

    def _loadNetworkParameters(self, saveToDir):
        filename = saveToDir+'/'+saveToDir[saveToDir.rfind('/')+1:]
        if self.loaded:
            saver = tf.train.import_meta_graph(filename+'.meta')
        else:
            saver = tf.train.Saver()
        saver.restore(self.sess, tf.train.latest_checkpoint(saveToDir))


class _Dataset:

    def __init__(self,x_data,y_data=None):
        if isinstance(x_data,list):
            x_data = np.asarray(x_data)

        self.index_in_epoch = 0
        self.x_data = x_data
        self.num_samples = x_data.shape[0]

        if y_data:
            if isinstance(y_data,list):
                y_data = self._convertY(y_data)
                self.y_data = y_data
        else:
            self.y_data = []
    def __len__(self):
        return len(self.x_data)

    def getClassCounts(self):
        return np.sum(self.y_data,axis=0)

    def _convertY(self, y_data):
        out = np.zeros((len(y_data),2))
        for i,cl in enumerate(y_data):
            out[i][cl] = 1
        return out

    def next_batch(self,batch_size):
        start = self.index_in_epoch
        end = self.index_in_epoch + batch_size

        if start == 0:
            idx = np.arange(0, self.num_samples)  # get all possible indexes
            np.random.shuffle(idx)  # shuffle indexes
            self.x_data = self.x_data[idx]
            if len(self.y_data) > 0:
                self.y_data = self.y_data[idx]

        if end < self.num_samples:
            self.index_in_epoch = end
            return self.x_data[start:end], self.y_data[start:end], False # epoch finished = False
        else:
            self.index_in_epoch = 0
            return self.x_data[start:], self.y_data[start:], True #epoch finished = True


    def stepsInEpoch(self,batch_size):
        return math.ceil(len(self) / batch_size)

    def getX(self):
        return self.x_data

    def getSequenceLength(self):
        return len(self.x_data[0])

Instructions for updating:
non-resource variables are not supported in the long term


In [47]:
def readInputs(f1,f2):
    lines_pos = open(f1).readlines()
    lines_neg = open(f2).readlines()

    X = []
    Y = []

    for l in convertLines(lines_pos):
        X.append(l)
        Y.append(1)
    for l in convertLines(lines_neg):
        X.append(l)
        Y.append(0)

    return X,Y

def convertLines(lines):
    v = []
    newLines = []
    for line in lines:
        newline = []
        for c in line.strip():
            if c == 'A':
                v = [1,0,0,0]
            elif c == 'C':
                v = [0,1,0,0]
            elif c == 'G':
                v = [0,0,1,0]
            elif c == 'T':
                v = [0,0,0,1]
            newline.append(v)
        newLines.append(newline)
    return newLines
def regular_network():
    net_model = NetworkModel()
    net_model.addInputLayer()
    net_model.addFullyConnectedLayer(50)
    net_model.addFullyConnectedLayer(50)
    net_model.addOutputLayer()
    return net_model

In [48]:
def regular_network():
    net_model = NetworkModel()
    net_model.addInputLayer()
    net_model.addFullyConnectedLayer(50)
    net_model.addFullyConnectedLayer(50)
    net_model.addOutputLayer()
    return net_model

In [60]:
rm = regular_network()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:81: UserWarning: `tf.layers.flatten` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Flatten` instead.
/usr/local/lib/python3.7/dist-packages/keras/legacy_tf_layers/core.py:523: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  return layer.apply(inputs)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:84: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
/usr/local/lib/python3.7/dist-packages/keras/legacy_tf_layers/core.py:255: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  return layer.apply(inputs)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:99: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future vers

In [53]:
X, y = readInputs('pos.txt', 'neg.txt')

In [55]:
from sklearn.model_selection import train_test_split

In [58]:
trainX, curX, trainY, curY = train_test_split(X, y, test_size=0.3, random_state=42)
testX, validX, testY, validY = train_test_split(curX, curY, test_size=0.1, random_state=42)

In [59]:
def conv_network():
    con_model = NetworkModel()
    con_model.addInputLayer()
    con_model.addConvLayer(10, 7)
    con_model.addMaxPoolLayer(5)
    con_model.addConvLayer(20, 5)
    con_model.addMaxPoolLayer(5)
    con_model.addFullyConnectedLayer(15)
    con_model.addOutputLayer()
    return con_model

200

In [61]:
My_Model = NetworkModel()
My_Model.addInputLayer()
My_Model.addConvLayer(10, 7)
My_Model.addMaxPoolLayer(5)
My_Model.addConvLayer(20, 5)
My_Model.addMaxPoolLayer(5)
My_Model.addFullyConnectedLayer(15)
My_Model.addOutputLayer()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:55: UserWarning: `tf.layers.conv1d` is deprecated and will be removed in a future version. Please Use `tf.keras.layers.Conv1D` instead.
/usr/local/lib/python3.7/dist-packages/keras/legacy_tf_layers/convolutional.py:288: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  return layer.apply(inputs)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:70: UserWarning: `tf.layers.max_pooling1d` is deprecated and will be removed in a future version. Please use `tf.keras.layers.MaxPooling1D` instead.
/usr/local/lib/python3.7/dist-packages/keras/legacy_tf_layers/pooling.py:294: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  return layer.apply(inputs)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:81: UserWarning: `tf.layers.flatten` is deprecated and will 

In [62]:
My_Model.printDetails()

####################################
Network information:
This network has 3757 trainable parameters.
 0. Input layer                                                                -> Output size: (?, 200, 4)
 1. Convolutional layer     10 filters, width 7, zero padding, with ReLU       -> Output size: (?, 200, 10)
 2. Max pooling layer       pool size 5                                        -> Output size: (?, 40, 10)
 3. Convolutional layer     20 filters, width 5, zero padding, with ReLU       -> Output size: (?, 40, 20)
 4. Max pooling layer       pool size 5                                        -> Output size: (?, 8, 20)
 5. Flatten layer                                                              -> Output size: (?, 160)
 6. Fully-connected layer   15 neurons, with ReLU                              -> Output size: (?, 15)
 7. Softmax (output) layer  2 neurons                                          -> Output size: (?, 2)

####################################


In [64]:
My_Model.train(trainX, trainY, validX, validY, 15)

####################################

Number of training examples: 7005
Distribution of the training set:
  # elements of class 0 = 6999
  # elements of class 1 = 6

Number of validation examples: 301
Distribution of the validation set:
  # elements of class 0 = 301
  # elements of class 1 = 0

 epoch |   train cost   |   valid cost   |  train acc  |  valid acc  |   time   
-------+----------------+----------------+-------------+-------------+----------
     0 |   0.30837816   |   0.99857247   |  0.3103329  | 1.0000000  | 0.00s 
     1 |   0.00758863   |   0.00013629   |  0.9991435  | 1.0000000  | 1.76s -> model selected
     2 |   0.00738818   |   0.00016736   |  0.9991435  | 1.0000000  | 1.47s 
     3 |   0.00691130   |   0.00033344   |  0.9991435  | 1.0000000  | 1.52s 
     4 |   0.00661097   |   0.00062330   |  0.9991435  | 1.0000000  | 1.45s 
     5 |   0.00650680   |   0.00090470   |  0.9991435  | 1.0000000  | 1.51s 
     6 |   0.00645361   |   0.00080487   |  0.9991435  | 1.0000